In [26]:
import pandas as pd
r_cols=['user_id','movie_id','rating']
ratings=pd.read_csv("C:\\Users\\Ghost\\Desktop\\LearningML\\materials\\ml-100k\\u_data.data",sep='\t',names=r_cols,usecols=range(3),encoding='latin-1')

m_cols=['movie_id','title']
movies=pd.read_csv("C:\\Users\\Ghost\\Desktop\\LearningML\\materials\\ml-100k\\u_item.item",sep='|',names=m_cols,usecols=range(2),encoding='latin-1')

ratings=pd.merge(movies,ratings)

print(ratings.head())

   movie_id             title  user_id  rating
0         1  Toy Story (1995)      308       4
1         1  Toy Story (1995)      287       5
2         1  Toy Story (1995)      148       4
3         1  Toy Story (1995)      280       4
4         1  Toy Story (1995)       66       3


In [27]:
#making matrices of user as rows movies as columns
userRatings=ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
print(userRatings.head())

title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
0                              NaN           NaN                    NaN   
1                              NaN           NaN                    2.0   
2                              NaN           NaN                    NaN   
3                              NaN           NaN                    NaN   
4                              NaN           NaN                    NaN   

title    12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                 
0                        NaN         NaN                          NaN   
1                        5.0         NaN                          NaN   
2                        NaN         NaN                          NaN   
3                        NaN         2.0                          NaN   
4                        NaN        

In [28]:
#now corr() will compute a correlation score for every column pair in matrix
corrMatrix=userRatings.corr()
print(corrMatrix.head())

title                      'Til There Was You (1997)  1-900 (1994)  \
title                                                                
'Til There Was You (1997)                        1.0           NaN   
1-900 (1994)                                     NaN           1.0   
101 Dalmatians (1996)                           -1.0           NaN   
12 Angry Men (1957)                             -0.5           NaN   
187 (1997)                                      -0.5           NaN   

title                      101 Dalmatians (1996)  12 Angry Men (1957)  \
title                                                                   
'Til There Was You (1997)              -1.000000            -0.500000   
1-900 (1994)                                 NaN                  NaN   
101 Dalmatians (1996)                   1.000000            -0.049890   
12 Angry Men (1957)                    -0.049890             1.000000   
187 (1997)                              0.269191             0.666667  

In [29]:
#now we have to avoid spuirous result
corrMatrix=userRatings.corr(method='pearson',min_periods=300)#can use other coeffeicints method and min values
print(corrMatrix.head())

title                      'Til There Was You (1997)  1-900 (1994)  \
title                                                                
'Til There Was You (1997)                        NaN           NaN   
1-900 (1994)                                     NaN           NaN   
101 Dalmatians (1996)                            NaN           NaN   
12 Angry Men (1957)                              NaN           NaN   
187 (1997)                                       NaN           NaN   

title                      101 Dalmatians (1996)  12 Angry Men (1957)  \
title                                                                   
'Til There Was You (1997)                    NaN                  NaN   
1-900 (1994)                                 NaN                  NaN   
101 Dalmatians (1996)                        NaN                  NaN   
12 Angry Men (1957)                          NaN                  NaN   
187 (1997)                                   NaN                  NaN  

In [30]:
#drop NAN
myRatings=userRatings.loc[0].dropna()#loc[0] user id 0
print(myRatings)

title
Empire Strikes Back, The (1980)    5.0
Gone with the Wind (1939)          1.0
Star Wars (1977)                   5.0
Name: 0, dtype: float64


In [31]:
#now finding recomendations for fictious user loc[0]
simCandidates=pd.Series()
for i in range(0,len(myRatings.index)):
    print("Adding sims for "+ myRatings.index[i] + "...")
    #Retrieve similar movies to this one that I rated
    sims=corrMatrix[myRatings.index[i]].dropna()
    #Now scale its similarity by how well I rated this movie
    sims=sims.map(lambda x: x*myRatings[i])
    #Add the score to the list of similarity candidates
    simCandidates=simCandidates.append(sims)
#Glance at our results so far:
print ("sorting...")
simCandidates.sort_values(inplace=True,ascending=False)
print (simCandidates.head(20))

Adding sims for Empire Strikes Back, The (1980)...
Adding sims for Gone with the Wind (1939)...
Adding sims for Star Wars (1977)...
sorting...
Empire Strikes Back, The (1980)              5.000000
Star Wars (1977)                             5.000000
Star Wars (1977)                             3.741763
Empire Strikes Back, The (1980)              3.741763
Return of the Jedi (1983)                    3.606146
Return of the Jedi (1983)                    3.362779
Raiders of the Lost Ark (1981)               2.693297
Raiders of the Lost Ark (1981)               2.680586
Indiana Jones and the Last Crusade (1989)    1.750535
Back to the Future (1985)                    1.374196
Star Trek: First Contact (1996)              1.277646
Independence Day (ID4) (1996)                1.243770
Godfather, The (1972)                        0.948875
Toy Story (1995)                             0.900099
Twelve Monkeys (1995)                        0.777652
Liar Liar (1997)                             0.

C:\Users\Ghost\AppData\Local\Temp\ipykernel_17604\612082476.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates=pd.Series()
C:\Users\Ghost\AppData\Local\Temp\ipykernel_17604\612082476.py:10: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simCandidates=simCandidates.append(sims)
C:\Users\Ghost\AppData\Local\Temp\ipykernel_17604\612082476.py:10: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simCandidates=simCandidates.append(sims)
C:\Users\Ghost\AppData\Local\Temp\ipykernel_17604\612082476.py:10: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simCandidates=simCandidates.append(sims)


In [32]:
#apparantly duplicates of movies came to be so summing up
simCandidates=simCandidates.groupby(simCandidates.index).sum()

In [33]:
simCandidates.sort_values(inplace=True,ascending=False)
print(simCandidates.head(40))

Empire Strikes Back, The (1980)              8.741763
Star Wars (1977)                             8.741763
Return of the Jedi (1983)                    6.968925
Raiders of the Lost Ark (1981)               5.373883
Indiana Jones and the Last Crusade (1989)    1.750535
Back to the Future (1985)                    1.374196
Star Trek: First Contact (1996)              1.277646
Independence Day (ID4) (1996)                1.243770
Godfather, The (1972)                        0.948875
Toy Story (1995)                             0.900099
Twelve Monkeys (1995)                        0.777652
Liar Liar (1997)                             0.751459
Contact (1997)                               0.674810
Rock, The (1996)                             0.239686
Fargo (1996)                                 0.222073
Silence of the Lambs, The (1991)             0.197439
Pulp Fiction (1994)                         -0.107838
dtype: float64


In [35]:
#filter movies I have watched

filteredSims=simCandidates.drop(myRatings.index)
#for ratings on 5
#for i in range(0,len(filteredSims.index)):
#    filteredSims[i]=filteredSims[i]/10*5
print(filteredSims.head())
#Somehow one movie left in that axis

KeyError: "['Gone with the Wind (1939)'] not found in axis"